### Подключение библиотек

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
import pymupdf
import io
import glob

# Парсинг сайта

#### Получение информации с сайта

In [2]:
# GET - запрос
url = 'https://store.steampowered.com/search/?filter=topsellers&ndl=1'
page = requests.get(url)
page.encoding = 'utf-8'

In [3]:
page.status_code

200

In [4]:
soup = bs(page.text, 'html.parser')

In [5]:
page.text

'<!DOCTYPE html>\n<html class=" responsive DesktopUI" lang="en">\n<head>\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\t\t<meta name="viewport" content="width=device-width,initial-scale=1">\n\t\t<meta name="theme-color" content="#171a21">\n\t\t<title>Steam Search</title>\n\t<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">\n\n\t\n\t\n\t<link href="https://store.fastly.steamstatic.com/public/shared/css/motiva_sans.css?v=YzJgj1FjzW34&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css">\n<link href="https://store.fastly.steamstatic.com/public/shared/css/shared_global.css?v=LkH7FFmHUNxE&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css">\n<link href="https://store.fastly.steamstatic.com/public/shared/css/buttons.css?v=ecbk11GZ8OUy&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css">\n<link href="https://store.fastly.steamstatic.com/public/css/v6/store.css?v=EF648k1z4gjy&amp;l=english&amp;_cdn=fastly" rel="st

In [6]:
soup

<!DOCTYPE html>

<html class="responsive DesktopUI" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Steam Search</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://store.fastly.steamstatic.com/public/shared/css/motiva_sans.css?v=YzJgj1FjzW34&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css"/>
<link href="https://store.fastly.steamstatic.com/public/shared/css/shared_global.css?v=LkH7FFmHUNxE&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css"/>
<link href="https://store.fastly.steamstatic.com/public/shared/css/buttons.css?v=ecbk11GZ8OUy&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css"/>
<link href="https://store.fastly.steamstatic.com/public/css/v6/store.css?v=EF648k1z4gjy&amp;l=english&amp;_cdn=fastly" rel="stylesheet" type="text/css"/>
<link h

In [7]:
result_list = {'title': [], 'rating': [], 'date': [], 'tags': [], 'description': []}

### Алгоритм

In [8]:
games = soup.find_all('a', class_='search_result_row')[:200]
for idx, game in enumerate(games, 1):
    title = game.find('span', class_='title').text.strip()
    if title.__contains__('Steam Deck'):
        continue
    print(title)
    result_list['title'].append(title)
    date = game.find('div', class_='search_released').text.strip()
    print(date)
    result_list['date'].append(date)
    game_url = game['href']
    print(game_url)
    game_page = requests.get(game_url)
    game_soup = bs(game_page.text, 'html.parser')
    description_tag = game_soup.find("div", {"id": "game_area_description"})
    description = description_tag.text.replace('\n', ' ').replace('\t','').replace('About This Game','').replace('About This Software','').strip() if description_tag else 'No description'
    print(description)
    result_list['description'].append(description)
    tags = ', '.join([tag.text.strip() for tag in game_soup.select('.glance_tags.popular_tags a')])
    print(tags)
    result_list['tags'].append(tags)
    if game_soup.find('div', class_='summary column').text.__contains__('No'):
        review_text = 'No user reviews'
    else:
        review_text = game_soup.find('meta', itemprop='ratingValue').attrs['content'] + ' / 10'
    print(review_text + "\n")
    result_list['rating'].append(review_text)

Counter-Strike 2
21 Aug, 2012
https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_7000_150_1
For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.A free upgrade to CS:GO, Counter-Strike 2 marks the largest technical leap in Counter-Strike’s history. Built on the Source 2 engine, Counter-Strike 2 is modernized with realistic physically-based rendering, state of the art networking, and upgraded Community Workshop tools.In addition to the classic objective-focused gameplay that Counter-Strike pioneered in 1999, Counter-Strike 2 features:All-new CS Ratings with the updated Premier modeGlobal and Regional leaderboardsUpgraded and overhauled mapsGame-changing dynamic smoke grenadesTick-rate-independent gameplayRedesigned visual effects and audioAll items from CS:GO moving forward to CS2
FPS, Shooter, Multiplay

In [9]:
print(len(result_list['title']))
print(len(result_list['rating']))
print(len(result_list['date']))
print(len(result_list['description']))
print(len(result_list['tags']))

49
49
49
49
49


### Создание датафрейма и запись в файл

In [10]:
file_name = 'games.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)
df.head()

,title,rating,date,tags,description
0,Counter-Strike 2,9 / 10,"21 Aug, 2012","FPS, Shooter, Multiplayer, Competitive, Action...","For over two decades, Counter-Strike has offer..."
1,The Elder Scrolls IV: Oblivion Remastered,9 / 10,"22 Apr, 2025","RPG, Open World, Fantasy, Singleplayer, Advent...",The Elder Scrolls IV: Oblivion™ Remastered mod...
2,Clair Obscur: Expedition 33,9 / 10,"24 Apr, 2025","Turn-Based Combat, Story Rich, Fantasy, Explor...","Once a year, the Paintress wakes and paints up..."
3,Schedule I,10 / 10,"24 Mar, 2025","Simulation, Co-op, Crime, Multiplayer, Open Wo...","Schedule I drugs, substances, or chemicals are..."
4,R.E.P.O.,10 / 10,"26 Feb, 2025","Horror, Online Co-Op, Comedy, Multiplayer, Co-...",R.E.P.O. is an online co-op horror game featur...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        49 non-null     object
 1   rating       49 non-null     object
 2   date         49 non-null     object
 3   tags         49 non-null     object
 4   description  49 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


# Парсинг PDF файла

In [49]:
# загрузка в list пути всех нужных pdf файлов
all_pdf = glob.glob(r"PDF2\*.pdf")

In [50]:
len(all_pdf)

18

In [51]:
all_pdf

['PDF2\\Cocoapods, Carthage, SPM как выбрать менеджер зависимостей в iOS.pdf',
 'PDF2\\Deutsche Telekom и Perplexity объявили о новом «AI Phone» стоимостью менее 1 000 долларов _ Хабр.pdf',
 'PDF2\\imgonline-com-ua-site2pdfB28eKGLy6EjA.pdf',
 'PDF2\\OSINT & Hacking — как работает фишинг для нельзяграма _ Хабр3.pdf',
 'PDF2\\Yandex N.V. запретили в\xa0течение пяти лет создавать аналогичные «Яндексу» сервисы _ Хабр.pdf',
 'PDF2\\Быстрое начало работы с Gitlab CICD.pdf',
 'PDF2\\Генеральный директор Mozilla покинула свой пост _ Хабр.pdf',
 'PDF2\\За что безопасники будут гореть в аду_ _ Хабр.pdf',
 'PDF2\\Заезжаем в Kotlin Multiplatform. Но какой ценой_ _ Хабр.pdf',
 'PDF2\\ИИ-агенты в Альфа-Банке_ нейросети создают автотесты без участия человека _ Хабр.pdf',
 'PDF2\\Инструменты наблюдаемости, о которых нужно знать в 2023 году.pdf',
 'PDF2\\История российской науки_ напишем вместе_ _ Хабр.pdf',
 'PDF2\\Как системному аналитику написать хорошее резюме —\xa011 рекомендаций _ Хабр.pdf',
 'PD

#### Получение информации с файла

In [52]:
for i in all_pdf:
    pdf_document = i
    doc = pymupdf.open(pdf_document)
    print("Исходный документ: ", doc)
    print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
    print(doc.metadata)
    for current_page in range(len(doc)):
        page = doc.load_page(current_page)
        page_text = page.get_text("text")
        print("Стр. ", current_page+1, "\n")
        print(page_text)

Исходный документ:  Document('PDF2\Cocoapods, Carthage, SPM как выбрать менеджер зависимостей в iOS.pdf')

Количество страниц: 29

------------------


{'format': 'PDF 1.7', 'title': 'Aspose', 'author': 'Aspose', 'subject': 'Aspose', 'keywords': '', 'creator': 'Aspose Pty Ltd.', 'producer': 'Aspose.PDF for .NET 23.2.0', 'creationDate': "D:20230320172822+00'00'", 'modDate': "D:20230320172822+00'00'", 'trapped': '', 'encryption': None}
Стр.  1 

117.94
Рейтинг
red_mad_robot
№1 в разработке цифровых решений для бизнеса
 6 часов назад
Средний18 мин
redmadrobot
Cocoapods, Carthage, SPM: как выбрать
менеджер зависимостей в iOS
179
Блог компании red_mad_robot
, 
Разработка под iOS*
FAQ
КАК СТАТЬ АВТОРОМ

Стр.  2 

Старший iOS-разработчик red_mad_robot Аня Кочешкова
рассказывает, чем отличаются три менеджера зависимостей, в каких
случаях и для каких задач подойдёт тот или иной. Материал будет
полезен джун-специалистам, которые только начали погружаться
в разработку: специально для них подробно

In [56]:
def extract_text_from_pdf(pdf):
    doc = pymupdf.open(pdf)
    text = ''
    for current_page in range(len(doc)):
        page = doc.load_page(current_page)
        page_text = page.get_text("text")
        text += page_text
    return text

In [57]:
def pars_pdf(text):
    text2 = text.split('\n')
    if 'Рейтинг' in text2:
        num = text2.index('Рейтинг')
        raiting = text2[num-1]
        activity = text2[num+2]
        date_publish = text2[num+3]
    if 'назад' in text2.lower():
        num3 = text2.index('азад')
        if (text2[num3+2][0].isnumeric()):
            name = text2[num3+1]
        else:
            name = text2[num3+1] + text2[num3+1]
    else:
        raiting = 0
        name = ""
        activity = ""
        if 'Простой' in text2:
            num2 = text2.index('Простой')
            date_publish = text2[num2-1]
        else:
            date_publish = ""
    
    return  name, raiting, activity, date_publish

In [59]:
Brak = []
NameCompany = []
Description = []
Raiting = []
Hubs = []
DataPublish = []
Activity = []
for pdf in all_pdf:
    try:  
        x = extract_text_from_pdf(pdf)
        #print(pdf.split('\\')[-1])
        N, R, A, D = pars_pdf(x)
        D2 = ''.join(x).replace('\n',' ')
        NameCompany.append(N)
        print(N)
        Raiting.append(R)
        print(R)
        DataPublish.append(D)
        print(D)
        Activity.append(A)
        print(A)
        Description.append(D2)
        print(D2)
    except:
        Brak.append(pdf)

### Алгоритм

In [19]:
all_names = []
all_descriptions = []
all_hubs = []
all_ratings = []
all_subscribers = []
def pars_pdf(text):
    text2 = text.split('\n')
    print(text2)
    for i in range(len(text2)-1, 0, -1):
        if text2[i].__contains__('Хабы:'):
            hubs = text2[i]
            if (text2[i+1].isnumeric() or text2[i+1].__contains__(".")) and (text2[i+2].__contains__("K") or text2[i+2].isdecimal()):
                rating = text2[i+1]
                subscribers = text2[i+2]
            elif (text2[i+2].isnumeric() or text2[i+2].__contains__(".")) and (text2[i+3].__contains__("K") or text2[i+3].isdecimal()):
                hubs += text2[i+1]
                rating = text2[i+2]
                subscribers = text2[i+3]
            else:
                for j in text2[i+2]:
                    if (j.isdigit()):
                        hubs += text2[i+2][0:text2[i+2].index(j)-1]
                        rating = text2[i+2][text2[i+2].index(j):-1]
                        subscribers = text2[i+3]
            description = text2[i-1]
            name = text2[i-2]
            print(f"\n\n\n{name}\n{description}\n{hubs}\n{rating}\n{subscribers}\n\n\n")
            text2.pop(i)
            text2.pop(i+1)
            text2.pop(i+2)
            if not ((text2[i+1].isnumeric() and (text2[i+2].__contains__("K") or text2[i+2].isnumeric())) and (text2[i+2].isnumeric() and (text2[i+3].__contains__("K") or text2[i+3].isnumeric()))):
                text2.pop(i+3)
            text2.pop(i-1)
            text2.pop(i-2)                    
            all_names.append(name)
            all_descriptions.append(description)
            all_hubs.append(hubs)
            all_ratings.append(rating)
            all_subscribers.append(subscribers)
pars_pdf(extract_text_from_pdf(r"PDF2\Компании _ Хабр.pdf"))
pars_pdf(extract_text_from_pdf(r"PDF2\Компании.pdf"))

['', 'Все потоки', 'Добавить компанию', 'RUVDS.com', 'VDS/VPS-хостинг. Скидка 15% по коду HABR15', 'Пишет в хабы:  Читальный зал,\u2004DIY или Сделай сам,\u2004Научно-популярное,\u2004Электроника для начинающих,\u2004', 'Системное администрирование', '2394.92', '72K', 'Selectel', 'IT-инфраструктура для бизнеса', 'Пишет в хабы:  Компьютерное железо,\u2004Гаджеты,\u2004IT-инфраструктура,\u2004IT-компании,\u2004', 'Производство и разработка электроники', '1935.27', '40K', 'Timeweb Cloud', 'Облачная платформа для разработчиков и бизнеса', 'Пишет в хабы:  Читальный зал,\u2004Научно-популярное,\u2004JavaScript,\u2004DIY или Сделай сам,\u2004История IT', '1538.54', '1.1K', 'OTUS', 'Цифровые навыки от ведущих экспертов', 'Пишет в хабы:  Программирование,\u2004Java,\u2004Машинное обучение,\u2004Тестирование веб-сервисов,\u2004Python', '830.05', '103K', 'Яндекс', 'Как мы делаем Яндекс', 'Пишет в хабы:  Машинное обучение,\u2004Open source,\u2004Высокая производительность,\u2004Программирование,\u

### Создание датафрейма

In [20]:
df = pd.DataFrame({
'NameCompany': all_names,
'Description': all_descriptions,
'Hubs': all_hubs,
'Rating': all_ratings,
'Subscribers': all_subscribers
})
df

,NameCompany,Description,Hubs,Rating,Subscribers
0,Southbridge,Обеспечиваем стабильную работу highload-проектов,"Пишет в хабы: IT-инфраструктура, DevOps, Сист...",273.3,47K
1,JUG Ru Group,Конференции для Senior-разработчиков,"Пишет в хабы: Конференции, Java, Тестирование...",276.72,41K
2,Хабр Карьера,Помогаем строить карьеру в IT,"Пишет в хабы: Карьера в IT-индустрии, Управле...",280.35,14K
3,TINKOFF,IT’s Tinkoff — просто о сложном,"Пишет в хабы: Разработка под Android, Машинно...",282.47,40K
4,Ozon Tech,Стремимся делать лучший e-commerce в России,"Пишет в хабы: Конференции, Высокая производит...",282.75,28K
5,Postgres Professional,Разработчик СУБД Postgres Pro,"Пишет в хабы: PostgreSQL, SQL, Администрирова...",305.42,76K
6,Альфа-Банк,Лучший мобильный банк по версии Markswebb,"Пишет в хабы: Конференции, Анализ и проектиро...",311.59,24K
7,AGIMA,Крупнейший интегратор digital-решений,"Пишет в хабы: Карьера в IT-индустрии, Управле...",336.45,519
8,Конференции Олега Бунина (Онтико),Профессиональные конференции для IT-разработчиков,"Пишет в хабы: Управление разработкой, Конфере...",360.51,58K
9,FirstVDS,Виртуальные и выделенные серверы в ДЦ в Москве,"Пишет в хабы: Научно-популярное, Системное ад...",398.02,24K


### Запись в файл

In [21]:
df.to_csv("companies.csv", index=False)
df2 = pd.read_csv("companies.csv")
df2

,NameCompany,Description,Hubs,Rating,Subscribers
0,Southbridge,Обеспечиваем стабильную работу highload-проектов,"Пишет в хабы: IT-инфраструктура, DevOps, Сист...",273.30,47K
1,JUG Ru Group,Конференции для Senior-разработчиков,"Пишет в хабы: Конференции, Java, Тестирование...",276.72,41K
2,Хабр Карьера,Помогаем строить карьеру в IT,"Пишет в хабы: Карьера в IT-индустрии, Управле...",280.35,14K
3,TINKOFF,IT’s Tinkoff — просто о сложном,"Пишет в хабы: Разработка под Android, Машинно...",282.47,40K
4,Ozon Tech,Стремимся делать лучший e-commerce в России,"Пишет в хабы: Конференции, Высокая производит...",282.75,28K
5,Postgres Professional,Разработчик СУБД Postgres Pro,"Пишет в хабы: PostgreSQL, SQL, Администрирова...",305.42,76K
6,Альфа-Банк,Лучший мобильный банк по версии Markswebb,"Пишет в хабы: Конференции, Анализ и проектиро...",311.59,24K
7,AGIMA,Крупнейший интегратор digital-решений,"Пишет в хабы: Карьера в IT-индустрии, Управле...",336.45,519
8,Конференции Олега Бунина (Онтико),Профессиональные конференции для IT-разработчиков,"Пишет в хабы: Управление разработкой, Конфере...",360.51,58K
9,FirstVDS,Виртуальные и выделенные серверы в ДЦ в Москве,"Пишет в хабы: Научно-популярное, Системное ад...",398.02,24K


# Парсинг json файла

#### Получение информации с файла

In [22]:
with open(r'Data\GooseGaming.json', 'r', encoding="utf8") as file:
    data = json.load(file)

### Алгоритм

In [23]:
all_ref_text = []
all_ref_day = []
all_ref_month = []
all_ref_time = []
all_ref_rate = []
all_ref_views = []
def parse_json(local_data):
    for i in local_data['refs']:
        all_ref_text.append(i[0])
        all_ref_day.append(i[1]['day'])
        all_ref_month.append(i[1]['month'])
        all_ref_time.append(i[1]['time'])
        all_ref_rate.append(i[2]['rate'])
        all_ref_views.append(i[2]['views'])
parse_json(data)

### Создание датафрейма

In [24]:
df = pd.DataFrame({
    'ref text': all_ref_text,
    'ref day': all_ref_day,
    'ref month': all_ref_month,
    'ref time': all_ref_time,
    'ref rate': all_ref_rate,
    'ref views': all_ref_views
})
df

,ref text,ref day,ref month,ref time,ref rate,ref views
0,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,2300
1,Из новостей на этой неделе: Unity сменила лого...,13,октября,19:44,6,2500
2,"На игровую консоль не очень похоже, скорее — н...",9,июня,16:18,41,8800
3,"Всем привет, в июне OTUS вновь запускает курс ...",30,мая,04:12,7,4000
4,Источник: knowyourmeme\n\r\nИгра-мем Untitled ...,31,октября,12:44,12,3700
5,"Похоже, сегодня все стали крякать чуть больше ...",10,октября,10:28,24,6100
6,"В Оксфорде наступает яркое утро четверга, и в ...",28,сентября,16:01,19,16000


### Запись в файл

In [25]:
df.to_csv("articles.csv", index=False)
df2 = pd.read_csv("articles.csv")
df2.head()

,ref text,ref day,ref month,ref time,ref rate,ref views
0,7 декабря состоялась церемония награждения лау...,8,декабря,09:30,6,2300
1,Из новостей на этой неделе: Unity сменила лого...,13,октября,19:44,6,2500
2,"На игровую консоль не очень похоже, скорее — н...",9,июня,16:18,41,8800
3,"Всем привет, в июне OTUS вновь запускает курс ...",30,мая,04:12,7,4000
4,Источник: knowyourmeme\n\r\nИгра-мем Untitled ...,31,октября,12:44,12,3700
